# Introducción

En el presente proyecto se busca poner en práctica los conocimientos adquiridos a lo largo del curso *Introducción a la Minería de Datos*, para ello se realizará un análisis de datos recogidos sobre *Matrículas de Educación Superior* entre los años 2010 y 2019. La fuente del set de datos corresponde al *Servicio de Información de Educación Superior* (SIES) y presenta un registro de 1.2 millones de matrículas aproximadamente por año, cada una de estas corresponde a una fila en el dataset.  Cada matrícula cuenta con 49 atributos, como por ejemplo: nombre de la sede, región de la sede, rango de edad del matriculado, entre otros que se detallan más adelante.

La relevancia de estos datos radica en que, al realizar un análisis sobre ellos, podríamos obtener información que ayude a mejorar el acceso a la educación superior. Lo anterior, a través de información respecto a la demanda de educación y el cómo ésta se distribuye en la sociedad, lo que permitiría atacar de forma eficiente sesgos o problemáticas sociales inherentes a este proceso. También nos permite observar rasgos característicos que llevan a la deserción en la educación superior y, por ende, facilitar la búsqueda de soluciones a estos problemas.

# Análisis Exploratorio

En esta sección se describe el proceso realizado para adaptar el conjunto de datos, con el fin de poder trabajar con ellos, y, además, el proceso a través del cual se logró entender los datos y así plantearse preguntas acorde. 

## Características del dataset inicial
- 10 archivos csv con información anual desde el 2010 al 2019
- Cada archivo cuenta con aproximadamente 1.2 millones de observaciones.
- 49 atributos por año.
-  29 nominales categóricos
- 3 nominales rankeadas
- 5 nominales arbitrarias
- 12 ordinales discretas

Esta información se obtuvo del documento disponible en la página de descarga de los datos, en él se describe cada columna y, en algunos casos, se indican los valores posibles. Además, para observar en detalle las columnas se utilizaron las siguientes funciones de la librería ``Pandas``;

In [ ]:
# DataSet corresponde al DataFrame de Pandas en donde se cargaron los datos de un mismo año
DataSet.info()
DataSet.describe()
DataSet.head(n=20)
for col in list(DataSet):
    DataSet[[col]].drop_duplicates()

## Limpieza de los datos

En este ámbito se busca homogeneizar el tipo de dato para cada columna y la forma en que se representa un valor nulo a lo largo de todos los datos. De esta manera, se reemplazan las distintas formas de un dato nulo a el valor ``Nan`` de la librería ``numpy`` y se transforman los tipos de datos de las columnas a un solo tipo para cada una. El código principal se muestra a continuación;

In [ ]:
import numpy as np
#dfX corresponde al dataframe con los datos del año X
d = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]

for i in range(len(d)):
    print(i)
    d[i] = d[i].replace(r'^\s+$', np.nan, regex=True)
    d[i]["anio_ing_carr_act"] = d[i]["anio_ing_carr_act"].replace(9995, np.nan, regex=True)
    d[i]["anio_ing_carr_act"] = d[i]["anio_ing_carr_act"].replace(0, np.nan, regex=True)
    d[i]["anio_ing_carr_ori"] = d[i]["anio_ing_carr_ori"].replace(9995, np.nan, regex=True)
    d[i]["FEC_NAC_ALU"] = d[i]["FEC_NAC_ALU"].replace("190001", np.nan, regex=True)
    d[i]["FEC_NAC_ALU"] = d[i]["FEC_NAC_ALU"].replace(190001, np.nan, regex=True)
    d[i] = d[i].replace("SIN INFORMACION", np.nan, regex=True)
    d[i][["valor_arancel"]] = d[i][["valor_arancel"]].astype(np.float64)
    name = f'{2010+i}'
    d[i].to_csv(name+'.csv',index=False)

## Correlación entre columnas

Desechamos ciertas columnas redundantes o con alta correlación, que a simple vista describen el mismo fenómeno con distintos niveles de detalle. Aún así, nos reservamos la posibilidad de utilizar cualquier columna que consideremos necesaria para futuros análisis. Este procedimiento, más que nada, es para hacernos una “*vista panorámica*” de la naturaleza de los datos, y con esto poder hacer preguntas más concretas.
 
Como en el dataset tenemos en su mayoría atributos nominales, con el propósito de observar alguna correlación entre ellos, los desplegamos mediante la vectorización “*One Hot Encoding*”, ésta consiste en representar el atributo como un vector de dimensión igual a la cantidad de valores distintos en dicho atributo, y asignar para cada valor un vector ortonormal a todos los demás. Luego de esto, fue posible generar una matriz de correlación con los valores desplegados de los siguientes atributos;
- ``nivel_carrera_2``: Tipo de grado académico entregado por la carrera.
- ``region_sede``: Región de la sede.
- ``area_conocimiento``
- ``tipo_inst_2``: Clasificación del tipo de institución
- ``acreditada_carr``: Situación de acreditación de la carrera o programa informada por la institución al 30 de abril del año del proceso. 
- ``acreditada_inst``: Situación acreditación de institución al 30 de abril del año en proceso.
- ``jornada``
- ``rango_edad`` 
- ``requisito_ingreso``
- ``tipo_plan_carr``: Distinción dada por el tipo plan que posee la carrera o programa.
- ``vigencia_carrera``: Tipo de vigencia.

El código a continuación permite obtener un heatmap de la correlación entre todos los años.

In [ ]:
import seaborn
df_all = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8, df9])
correlation = df_all.corr()
seaborn.heatmap(correlation)

Por otro lado, a continuación se muestra el código con el cual se aplica *One Hot* a las columnas del año 2011, 2015 y 2019. Aquí, primero se obtienen los tipos de cada columa en un dataframe, luego, para cada columna, se verfica si su tipo es `object` y si no corresponde a columnas con datos nominales arbitrarios o presentes en un solo año. Así, a las columnas que pasan el filtro se les aplica *One Hot* y se guarda su resultado.

In [ ]:
df_sub = pd.concat([df1, df5, df9])

# Se obtienen los tipos de cada columa en un dataframe
df_sub_datatypes = (pd.DataFrame(df_sub.dtypes)
                    .reset_index()
                    .rename(columns = {0:'tipo', 'index' : 'columna'})
                    )

# para cada columna...
cols_name = list(df_sub)
init = cols_name.index("costo_proceso_titulacion") + 1
for i in range(init, len(df_sub_datatypes)):
    # Si es que es de tipo object...
    if str(df_sub_datatypes.iloc[i][1]) == 'object':
        col_name = df_sub_datatypes.iloc[i][0]
        # y no corresponde a una de las siguientes...
        if (col_name in ["codigo_unico", "nomb_inst", "nomb_sede", "nomb_carrera", "acre_inst_desde_hasta"]
            or col_name in ["costo_obtencion_titulo_diploma", "costo_tit_explicacion_observacion", "costo_proceso_titulacion"]) :
            continue
        print("iniciando proceso para " + col_name)
        cols_name.remove(col_name)
        # se le aplica la transformación One Hot y se guarda.
        df_one_hot = pd.get_dummies(df_sub[[col_name]], prefix=col_name)
        print("guardando")
        df_one_hot.to_csv("one_hot/one_hot_" + col_name + ".csv", index = False)
        print("done")
print(cols_name)

En el bloque a continuación se seleccionan las columnas a utilizar del dataframe  que no requieren ser transformadas, luego, se seleccionan las columnas que si requieren transformación y se quieren utilizar en la correlación. Entre ambos conjuntos se hace merge para obtener un solo DataFrame con toda la información.

In [ ]:
cols_to_drop = ["costo_obtencion_titulo_diploma", "costo_tit_explicacion_observacion", "costo_proceso_titulacion", "forma_ingreso", "nivel_global", "nivel_carrera_1", "comuna_sede", "provincia_sede", "area_carrera_generica", "oecd_area", "oecd_subarea", "tipo_inst_1", "tipo_inst_3", "modalidad"]
df_sub_one_hot = df_sub.drop(columns=cols_to_drop)

one_hot_cols = ["nivel_carrera_2", "region_sede", "area_conocimiento", "tipo_inst_2", "acreditada_carr", "acreditada_inst", "jornada", "rango_edad", "requisito_ingreso", "tipo_plan_carr", "vigencia_carrera"]
for col in one_hot_cols:
    print(col)
    df_aux = pd.read_csv("one_hot/one_hot_" + col + ".csv")
    df_sub_one_hot = pd.merge(df_sub_one_hot, df_aux, left_index=True, right_index=True)

Sobre el DataFrame obtenido se busca la correlación y el resultado se muestra en un HeatMap.

In [ ]:
import seaborn

corr_sub = df_sub_one_hot.corr()
seaborn.set(rc={'figure.figsize':(16,16)})
seaborn.heatmap(corr_sub)

## Comportamientos de los datos

Como primera visualización es interesante analizar la distribución del costo de los aranceles entre las distintas regiones del país, como también el incremento de estos a lo largo de los años; para lo anterior se transforman los datos agrupando por rango de edad, genero y área de conocimiento, luego se cuenta la cantidad de aparaciones de cada triplete por año, por último se promedian los valores encontrados por año.

In [ ]:
# se analiza por año y luego se promedia
d = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]
h1 = []
i = 2010

for df in d:
    dfh = (df.query("area_conocimiento!='Sin área definida'")[["GEN_ALU","rango_edad", "area_conocimiento"]]
        .groupby(by=["GEN_ALU","rango_edad", "area_conocimiento"])
        .size()
        .reset_index()
        .rename(columns = {0:'total'})
        )
    dfh["anio"] = [i]*len(dfh)
    i+=1

    h1.append(dfh.copy())

#concatenación y mapeo
dfh1 = (pd.concat(h1)[["GEN_ALU","rango_edad", "area_conocimiento", "total"]]
                 .groupby(by=["GEN_ALU","rango_edad", "area_conocimiento"])
                 .mean()
                 .reset_index()
                )
dfh1[["GEN_ALU"]] = dfh1[["GEN_ALU"]].replace({1:"Hombre", 2:"Mujer"})
dfh1 = (dfh1.replace({"SIN INFORMACION": "Sin información", "Ciencias Sociales": "C. Sociales"}))

Así se obtiene la siguiente figura utilizando D3.js. Esta responde, finalmente, a la pregunta **¿Cómo se relaciona el área de conocimiento de la carrera con la edad y género de la persona que se matricula?**.

Para continuar comprendiendo los datos, nos planteamos la pregunta 
**¿Existe una correlación entre el costo de las carreras y su ubicación geográfica? ¿Varía esto a través de los años?**, para ello se selecciona el *código de la carrera*, *el valor del arancel* y *la región de la sede*, se eliminan los duplicados y se agrupan por región para obtener el promedio de los valores de arancel de las diferentes carreras. Lo anterior se hace para los años 2011, 2013, 2015, 2017 y 2019. Por último, se normaliza según la UF de cada año comprendiendo que los datos originales están en pesos chilenos y esta moneda se ve afectada por la inflación anualmente.
 
Los datos finales se expotaron al software Tableu para graficar.

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

d = [df1, df3, df5, df7, df9]
h2 = []
i = 2011

for df in d:
    try:
        dfh = df.query("vigencia_carrera!='NO VIGENTE'")
    except:
        dfh = df
    dfh = (dfh[["codigo_unico","valor_arancel", "region_sede"]]
           .drop_duplicates()[["region_sede", "valor_arancel"]]
           .groupby(["region_sede"])
           .mean()
           .reset_index()
           .rename(columns = {"valor_arancel":'arancel_promedio_pesos'})
        )
    dfh["anio"] = [i]*len(dfh)
    i+=1
    h2.append(dfh.copy())

df_arancel_original = pd.concat(h2)

df_UF = pd.read_csv("UF.csv").rename(columns={"Periodo": "anio"})
df_UF["UF"] = df_UF["UF"].astype(float)
df_arancel_original = pd.merge(df_arancel_original, df_UF, how = "inner", on = ["anio"])
df_arancel_original["arancel_promedio_UF"] = df_arancel_original.arancel_promedio_pesos/df_arancel_original.UF
df_arancel_original = df_arancel_original[["anio", "region_sede", "arancel_promedio_UF"]].sort_values("anio", ascending = False)

df_arancel_mapeado = (df_arancel_original.replace({'Arica y Parinacota': 1, 'Tarapacá': 2, 'Antofagasta': 3, 'Atacama': 4, 'Coquimbo': 5, 'Valparaíso': 6, 'Metropolitana': 7, "Lib. Gral B. O'Higgins": 8, 'Maule': 9, 'Ñuble': 10, 'Biobío': 11, 'La Araucanía': 12, 'Los Ríos': 13, 'Los Lagos': 14, 'Aysén': 15, 'Magallanes': 16}))
df_arancel_mapeo_regiones = pd.DataFrame({"Region": ['Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama', 'Coquimbo', 'Valparaíso', 'Metropolitana', "Lib. Gral B. O'Higgins", 'Maule', 'Ñuble', 'Biobío', 'La Araucanía', 'Los Ríos', 'Los Lagos', 'Aysén', 'Magallanes'], "Equivalente": list(range(1,17))})

Se extrae de la imagen que el valor (promedio) de los aranceles varían por región, siendo el valor más alto en la Región Metropolitana. También al analizar por año, se ve un incremento en Unidades de Fomento (UFs) del valor de los aranceles.

# Preguntas
1. ¿Se puede predecir las características de las matrículas para años futuros considerando los datos actuales? 
2. ¿Cuáles son las características y tendencias de las personas que se cambian de carrera? 
3. ¿Cuánto valor le agrega la acreditación a una carrera?
4. ¿Existe una relación entre costo de carrera y otras variables (numéricas?)
 
Siendo conscientes de que estamos trabajando mayoritariamente con variables categóricas, puede ser posible encontrar relaciones no lineales entre los atributos al vectorizarlos de manera más compleja que la entregada por *one hot encoding*, tal vez mediante vectores que comprendan cierta relación entre los valores del atributo, que posean cierta métrica (*word2vec* por ejemplo). De esta forma podemos permitirnos generar regresiones, predicciones o clustering más ricos en información relevante.


# Anexos
Descripción del dataset dado por el Mineduc [aquí](http://datos.mineduc.cl/datasets/178941-er-base-de-datos-matricula-en-educacion-superior.download/).

A continuación se muestra el código utilizado para obtener los gráficos relativos a la cantidad de datos anuales y al porcentaje de datos nulos en cada año.

In [ ]:
d = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]
nulos, l, l_total, anno, porcentaje = [], [], [], [], []
i = 2010

for df in d:
    n = 0
    for v in list(df.isnull().sum()):
        n += v
    anno.append(i)
    nulos.append(n)
    t = len(df)*len(list(df))*1.0
    l.append(len(df))
    porcentaje.append((n/t)*100)
    i+=1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize'] = [11, 5.5]


fig_total_datos = plt.figure()
fig_total_datos.clf()
ax_total_datos = fig_total_datos.add_subplot(111)
#ax_total_datos.bar(anno, l, align='center', alpha=0.7, color = 'darkorange', label = "Entradas en la tabla")
ax_total_datos.plot(anno, l, alpha=1, color = '#00d4f0', label = "Entradas en la tabla")
ax_total_datos.plot(anno, l, 'o', alpha=1, color = '#071554', label = "Entradas en la tabla")
ax_total_datos.set_ylabel("Total de datos")
ax_total_datos.set_xlabel("Año de los datos")
ax_total_datos.set_title("Cantidad de datos recolectados para cada año")
#set off Scientific notation from Y-axis
ax_total_datos.get_yaxis().get_major_formatter().set_useOffset(False)
ax_total_datos.get_yaxis().get_major_formatter().set_scientific(False)
#ax_total_datos.legend(loc="best")
fig_total_datos.savefig("total_datos.png", transparent=False)
fig_total_datos.show()


fig_porcentaje_nulo = plt.figure()
fig_porcentaje_nulo.clf()
ax_porcentaje_nulo = fig_porcentaje_nulo.add_subplot(111)
graph = ax_porcentaje_nulo.bar(anno, porcentaje, align='center', color = '#071554', label = "Porcentaje de datos nulos")
ax_porcentaje_nulo.bar([2016], [14], align='center', alpha=0)
ax_porcentaje_nulo.set_ylabel("Porcentaje respecto al total de datos")
ax_porcentaje_nulo.set_xlabel("Año de los datos")
ax_porcentaje_nulo.set_title("Porcentaje de datos nulos en los datos recolectados para cada año")
#fig_porcentaje_nulo.legend(loc="upper left")
i = 0
for p in graph:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()

    num = str(porcentaje[i]).split(".")
    t = num[0] +'.' + num[1][:2]+'%'
    plt.text(x+width/2,
             y+0.5+height*1,
             t,
             ha='center')
             #weight='bold')
    i+=1
fig_porcentaje_nulo.savefig("porcentaje_datos_nulos.png", transparent=False)
fig_porcentaje_nulo.show()